In [1]:
import cv2,os
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,precision_score,accuracy_score,roc_curve
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,random_split,DataLoader,SubsetRandomSampler
from torchvision import transforms
import torchvision.models as models

from vit_pytorch import ViT
from vit_pytorch.mobile_vit import MobileViT

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("torch.device(cuda)")
    print("torch.cuda.device_count(): ", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(torch.cuda.get_device_name())
    print("torch.cuda.current_device()", torch.cuda.current_device())
else:
    device = torch.device("cpu")
    print("torch.device(cpu)")

torch.device(cuda)
torch.cuda.device_count():  4
Tesla V100-SXM2-16GB
Tesla V100-SXM2-16GB
Tesla V100-SXM2-16GB
Tesla V100-SXM2-16GB
torch.cuda.current_device() 0


# 1. Read data

In [3]:
chip="All"
X_Ctrl = np.load("./Datasets/Ctrl_"+chip+".npy",allow_pickle=True)[:1000]
X_VPA = np.load("./Datasets/VPA_"+chip+".npy",allow_pickle=True)[:1000]
y_Ctrl = torch.zeros(len(X_Ctrl), dtype=torch.int64)
y_VPA = torch.ones(len(X_VPA), dtype=torch.int64)
X = np.concatenate((X_Ctrl, X_VPA), axis = 0)
y = torch.cat((y_Ctrl, y_VPA), 0)

# 2. Data processing

In [4]:
class cell_dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.transform = transforms.ToTensor()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.transform(self.x[idx]).to(torch.float), F.one_hot(self.y[idx],num_classes=2).to(torch.float)

dataset = cell_dataset(X, y)

In [5]:
batch_size = 4
train_size = int(len(X)*0.8)
valid_size = len(X) - train_size

train_data, valid_data = random_split(dataset=dataset, lengths=[train_size, valid_size], 
                                      generator=torch.Generator().manual_seed(42))
dataloader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)
dataloader_valid = DataLoader(valid_data, batch_size=batch_size, shuffle=True)

In [6]:
train_data_01 = 0
train_size = len(train_data)
for i in range(train_size):
    train_data_01+=int(train_data[i][1][1].item())
print("Total number of train : ", train_size)
print("train_class_0: ", train_size-train_data_01)
print("train_class_1: ", train_data_01)

valid_data_01 = 0
valid_size = len(valid_data)
for i in range(valid_size):
    valid_data_01+=int(valid_data[i][1][1].item())
print("\nTotal number of valid : ", valid_size)
print("valid_class_0 num : ", valid_size-valid_data_01)
print("valid_class_1 num : ", valid_data_01)

Total number of train :  1600
train_class_0:  793
train_class_1:  807

Total number of valid :  400
valid_class_0 num :  207
valid_class_1 num :  193


# 3. Vision Transformer

In [7]:
model = ViT(
    image_size = 600,
    patch_size = 10,
    num_classes = 2,
    dim = 1024,
    depth = 1,
    heads = 2,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
).to(device)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
    
# #Freeze model weights
# for param in model.parameters():
#     param.requires_grad = True
# for param in model.fc.parameters():
#     param.requires_grad = True

In [8]:
image_size = 600
test_input = torch.ones(1,3,image_size,image_size).to(device)
print(model(test_input).size())

torch.Size([1, 2])


# 4. Train

In [9]:
model.avgpool = nn.AdaptiveAvgPool2d(1)
loss_function = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

def train(model,device,dataloader_train,loss_function,optimizer):
    losses_train = []
    n_train = 0
    acc_train = 0
    optimizer.step()
    model.train()
    for x, y in dataloader_train:
        n_train += y.size()[0]
        model.zero_grad()  # 勾配の初期化
        x = x.to(device)  # テンソルをGPUに移動
        y = y.to(device)
        output = model.forward(x)  # 順伝播
        loss = loss_function(output, y)  # 誤差(クロスエントロピー誤差関数)の計算
        loss.backward()  # 誤差の逆伝播
        optimizer.step()  # パラメータの更新
        acc_train += (output.argmax(1) == y[:,1]).float().sum().item()
        losses_train.append(loss.tolist())
    return np.mean(losses_train), (acc_train/n_train)
        
def valid(model,device,dataloader_valid,loss_function):
    losses_valid = []
    n_val = 0
    acc_val = 0
    model.eval()
    for x, y in dataloader_valid:
        n_val += y.size()[0]
        x = x.to(device)  # テンソルをGPUに移動
        y = y.to(device)
        output = model.forward(x)  # 順伝播
        loss = loss_function(output, y)  # 誤差(クロスエントロピー誤差関数)の計算
        acc_val += (output.argmax(1) == y[:,1]).float().sum().item()
        losses_valid.append(loss.tolist())
    return np.mean(losses_valid), (acc_val/n_val)

history = {'loss_train': [], 'loss_valid': [],'acc_train':[],'acc_valid':[]}

In [10]:
n_epochs = 10
for epoch in range(n_epochs):
    loss_train, acc_train = train(model,device,dataloader_train,loss_function,optimizer)
    loss_valid, acc_valid = valid(model,device,dataloader_valid,loss_function)
    scheduler.step()
    
    history['loss_train'].append(loss_train)
    history['loss_valid'].append(loss_valid)
    history['acc_train'].append(acc_train)
    history['acc_valid'].append(acc_valid) 
    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'
          .format(epoch, loss_train, acc_train, loss_valid, acc_valid))

../aten/src/ATen/native/cuda/Loss.cu:129: operator(): block: [0,0,0], thread: [0,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:129: operator(): block: [0,0,0], thread: [4,0,0] Assertion `input_val >= zero && input_val <= one` failed.


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
# train processing plot
n_epochs = 10
epochs=range(1,n_epochs+1)
plt.ylim(0,1.0)
plt.plot(epochs, history['acc_train'], 'b', label='Training accuracy')  
plt.plot(epochs, history['acc_valid'], 'r', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()
plt.show()

# Test data

In [10]:
# Test
X = X_test
y = y_test
y_pred = []
out_pred = []
total = X.shape[0]

model.eval()
for n in range(total):
    img = X[n]
    label = y[n]
    transform = transforms.Compose([transforms.ToTensor()])
    input_tensor = transform(img).unsqueeze(0).to(device)
    output = model(input_tensor)
    pred = output.argmax(1).cpu().item()
    out_pred.append(output[0][1].item())
    y_pred.append(pred)

y_pred = np.array(y_pred)
out_pred = np.array(out_pred)
print("total_test: {:}" .format(total))
print('accuracy_score: {:.3f}'.format(accuracy_score(y,y_pred)))
print('precision_score: {:.3f}'.format(precision_score(y,y_pred)))
print('roc_auc_score: {:.3f}'.format(roc_auc_score(y, out_pred)))

fpr, tpr, thresholds = roc_curve(y, out_pred,drop_intermediate=True)
plt.plot(fpr, tpr, marker='.')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()

# 99. Save model

In [35]:
for param in model.parameters():
    param.requires_grad = True
torch.save(model,"SelfResnet18_L2F1_250.pkl")